In [1]:
import networkx as nx
import json
from textblob import TextBlob
from datetime import datetime
from community import community_louvain
import matplotlib.pyplot as plt
import numpy as np

In [2]:
data = []
json_data=open('path\\SaudiArabia.json').read()
data.append(json.loads(json_data))


In [3]:

UserData = {}

for l in range(len(data)):
    for i in range(len(data[l])):
        for j in range(len(data[l][i])):
            obj = data[l][i][j]
            hashtags = {}
            if 'user' in obj:
                testimonial = TextBlob(obj['full_text'])
                if obj['user']['screen_name'] in UserData:
                    hashtags = UserData[obj['user']['screen_name']]
                for h in obj['entities']['hashtags']:
                    if not '#' + h['text'] in hashtags:
                        hashtags['#' + h['text']] = {}
                        hashtags['#' + h['text']]['tweeted'] = obj['created_at']
                        hashtags['#' + h['text']]['sentiment'] = testimonial.sentiment
                UserData[obj['user']['screen_name']] = hashtags

In [4]:
i = 0
for user in UserData:
    if i == 30:
        break
    print(user)
    print(json.dumps(UserData[user],indent=1))
    i += 1

LAmaleCA
{
 "#SaudiArabia": {
  "tweeted": "Wed Nov 21 01:13:18 +0000 2018",
  "sentiment": [
   0.9765625,
   0.5
  ]
 },
 "#IvankasEmails": {
  "tweeted": "Wed Nov 21 01:13:18 +0000 2018",
  "sentiment": [
   0.9765625,
   0.5
  ]
 },
 "#IvankaTrumpLockHerUp": {
  "tweeted": "Wed Nov 21 01:13:18 +0000 2018",
  "sentiment": [
   0.9765625,
   0.5
  ]
 },
 "#WhitakerMustRecuse": {
  "tweeted": "Wed Nov 21 01:13:18 +0000 2018",
  "sentiment": [
   0.9765625,
   0.5
  ]
 },
 "#Comey": {
  "tweeted": "Wed Nov 21 01:13:18 +0000 2018",
  "sentiment": [
   0.9765625,
   0.5
  ]
 },
 "#Clinton": {
  "tweeted": "Wed Nov 21 01:13:18 +0000 2018",
  "sentiment": [
   0.9765625,
   0.5
  ]
 },
 "#hillaryclinton": {
  "tweeted": "Wed Nov 21 01:13:18 +0000 2018",
  "sentiment": [
   0.9765625,
   0.5
  ]
 }
}
janicenall
{
 "#SaudiArabia": {
  "tweeted": "Wed Nov 21 01:13:15 +0000 2018",
  "sentiment": [
   -0.2,
   0.7
  ]
 }
}
NewfoundStudio
{
 "#SaudiArabia": {
  "tweeted": "Wed Nov 21 01:13:11 +0

In [5]:
def getSentimentArea(sentiment):
    if sentiment[0] == 0 and sentiment[1] == 0:
        return 1
    if sentiment[0] == 0 and sentiment[1] > 0:
        return 2
    if sentiment[0] > 0 and sentiment[1] > 0:
        return 3
    if sentiment[0] > 0 and sentiment[1] == 0:
        return 4
    if sentiment[0] == 0 and sentiment[1] < 0:
        return 5
    if sentiment[0] < 0 and sentiment[1] < 0:
        return 6
    if sentiment[0] < 0 and sentiment[1] == 0:
        return 7
    if sentiment[0] < 0 and sentiment[1] > 0:
        return 8
    if sentiment[0] > 0 and sentiment[1] < 0:
        return 9
    
    return 10

In [6]:
def calcHashtagWeight(hashtag1,hashtag2):
    fmt = '%a %b %d %H:%M:%S +0000 %Y'
    d1 = datetime.strptime(hashtag1['tweeted'], fmt)
    d2 = datetime.strptime(hashtag2['tweeted'], fmt)

    minutesDiff = 20000.0 - ((d1 - d2).seconds/60.0)
    
    s1 = getSentimentArea(hashtag1['sentiment'])
    s2 = getSentimentArea(hashtag2['sentiment'])
    
    if s1!=s2: #if the sentiment area is different then consider not connected. Optionally we could reduce the weight by a factor
        minutesDiff = 0
        
    return minutesDiff

In [7]:
def calcWeight(user1,user2):
    weight=0
    for hashtag in user1:
        if hashtag in user2:
            weight+=calcHashtagWeight(user1[hashtag],user2[hashtag])
    return weight

In [8]:
gephiData = []
i=-1
j=-1
for user1 in UserData:
    i=i+1
    j=-1
    if i%100==0:
        print("Processing please wait ... {0}%".format(i*100.0/len(UserData)))
    for user2 in UserData:
        j=j+1
        if j>i:
            weight = calcWeight(UserData[user1],UserData[user2])
            if weight > 0:
                edge = []
                edge.append(user1)
                edge.append(user2)
                edge.append('undirected')
                edge.append(weight)
                gephiData.append(edge)

np.savetxt("path\\SaudiArabia_edges.csv", gephiData, delimiter=",", fmt='%s', header="Source,Target,Type,Weight",comments='')
print('Done')

Processing please wait ... 0.0%
Processing please wait ... 8.952551477170994%
Processing please wait ... 17.905102954341988%
Processing please wait ... 26.85765443151298%
Processing please wait ... 35.810205908683976%
Processing please wait ... 44.76275738585497%
Processing please wait ... 53.71530886302596%
Processing please wait ... 62.66786034019696%
Processing please wait ... 71.62041181736795%
Processing please wait ... 80.57296329453894%
Processing please wait ... 89.52551477170994%
Processing please wait ... 98.47806624888094%
Done


In [10]:
def getTweets(username):
    for l in range(len(data)):
        for i in range(len(data[l])):
            for j in range(len(data[l][i])):
                obj = data[l][i][j]
                if 'user' in obj:
                    if obj['user']['screen_name'] == username:
                        print("User: {0} Tweet: {1}".format(username,obj['full_text']))
                    

In [15]:
getTweets('arabnews')
print(UserData['arabnews'])

User: arabnews Tweet: #SaudiArabia's King Salman arrives in Jouf as part of his tour of various regions of the Kingdom https://t.co/V4OdPqGERK
User: arabnews Tweet: #UPDATE: Donald #Trump says he could meet with #SaudiArabia's crown prince at the #G20 in #Argentina after saying the #US will remain a 'steadfast' partner of the Kingdom

https://t.co/8XSSVcKts6 https://t.co/KaXYEyMTvZ
{'#SaudiArabia': {'tweeted': 'Tue Nov 20 21:51:57 +0000 2018', 'sentiment': Sentiment(polarity=0.0, subjectivity=0.5)}, '#UPDATE': {'tweeted': 'Tue Nov 20 20:49:16 +0000 2018', 'sentiment': Sentiment(polarity=0.4, subjectivity=0.8)}, '#Trump': {'tweeted': 'Tue Nov 20 20:49:16 +0000 2018', 'sentiment': Sentiment(polarity=0.4, subjectivity=0.8)}, '#G20': {'tweeted': 'Tue Nov 20 20:49:16 +0000 2018', 'sentiment': Sentiment(polarity=0.4, subjectivity=0.8)}, '#Argentina': {'tweeted': 'Tue Nov 20 20:49:16 +0000 2018', 'sentiment': Sentiment(polarity=0.4, subjectivity=0.8)}, '#US': {'tweeted': 'Tue Nov 20 20:49:16 